In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from nltk.parse.generate import generate
from nltk import CFG
from nltk.grammar import Nonterminal

from eig.battleship import Parser

from battleship.grammar import BattleshipCFG

# Battleship grammar

In [8]:
cfg = BattleshipCFG(include_lambdas=False)

In [9]:
# Check that all generated programs are valid
valid, invalid = [], []

for tokens in generate(cfg, n=int(1e6), depth=5, start=Nonterminal('A')):
    program = ' '.join(tokens)
    try:
        Parser.parse(program, enforce_type=False)
        valid.append(program)
    except:
        invalid.append(program)
        print(program, flush=True)

print('Valid programs:', len(valid))
print('Invalid programs:', len(invalid))

In [ ]:
# Pass `enforce_type=False` to `Parser.parse` to allow all toplevel types
program = Parser.parse("( and TRUE ( touch Red Blue ) )", enforce_type=False)

In [ ]:
program.to_dict()